In [ ]:
!pip install wget
!pip install transformers-phobert
!pip install fastBPE
!pip install fairseq
!pip install vncorenlp
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=c0e9dd0953fcf9fd0e2d8a69cc9f81bfb7cc489893e7507274e27d5e9654ecc1
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 777 kB 20.5 MB/s 
     |████████████████████████████████| 3.0 MB 59.0 MB/s 
     |████████████████████████████████| 1.2 MB 55.4 MB/s 
     |████████████████████████████████| 880 kB 41.3 MB/s 
  Created wheel for transformers-phobert: filename=transformers_phobert-3.1.2-py3-none-any.whl size=937543 sha256=9ece1b3e3bd6d9aabb42a659f487f044b20916e6891e93ba92843d07242e20f0
  Stored in directory: /root/.cache/pip/wheels/41/4f/b0/793888110a36e99d07b5e8921aeb612d14566d721abb6c296c
  Created 

In [ ]:
import wget

In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2022-08-05 23:41:11--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.08s   

2022-08-05 23:41:11 (338 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-08-05 23:41:11--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting respon

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
# rdrsegmenter = VnCoreNLP("/content/drive/My Drive/BERT/SA/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [ ]:
with open('/content/cmt_2.txt', "r", encoding="UTF-8") as f:
    sentences_1 = f.readlines()
with open('/content/cmt_val.txt', "r", encoding="UTF-8") as f:
    sentences_2 = f.readlines()
# with open('/content/val2.txt', "r", encoding="UTF-8") as f:
#     sentences_3 = f.readlines()

In [ ]:
import re
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
 
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\\p{P}*)([p{L}.]*\\p{L}+)(\\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

In [ ]:
def combineToList(array):
  counter_happy = 0
  counter_sad = 0
  counter_angry = 0
  counter_surpise = 0
  counter_scare = 0
  counter_info = 0
  new_sen = []
  array_for_label_happy=[]
  array_for_label_sad=[]
  array_for_label_fury=[]
  array_for_label_surprise=[]
  array_for_label_scare=[]
  array_for_label_info=[]
  for i in array:
    i = i.replace('\n','').strip().lower()
    if i.strip() == '' or i.strip() =='.' or i.strip() =='...' or i.strip() =='?' or i.strip() =='??' or i.strip() =='love mom':
        continue
    if 'http' in i:
        continue
    else:
        i = i.split('\t')
        if len(i) == 2:
            if i[1] == 'hạnh phúc':
                array_for_label_happy.append(i)
                counter_happy += 1
            if i[1] == 'buồn bã':
                array_for_label_sad.append(i)
                counter_sad += 1
            if i[1] == 'phẫn nộ':
                array_for_label_fury.append(i)
                counter_angry += 1
            if i[1] == 'ngạc nhiên':
                array_for_label_surprise.append(i)
                counter_surpise += 1
            if i[1] == 'sợ hãi':
                array_for_label_scare.append(i)
                counter_scare += 1
            if i[1] == 'thông tin':
                array_for_label_info.append(i)
                counter_info += 1
  new_sen.extend(array_for_label_happy)
  new_sen.extend(array_for_label_sad)
  new_sen.extend(array_for_label_fury)
  new_sen.extend(array_for_label_surprise)
  new_sen.extend(array_for_label_scare)
  new_sen.extend(array_for_label_info)
  return new_sen

In [ ]:
array_for_label_happy=[]
array_for_label_sad=[]
array_for_label_fury=[]
array_for_label_surprise=[]
array_for_label_scare=[]
array_for_label_info=[]
new_sen2 = []
for i in sentences_2:
  i = i.replace('\n','').strip().lower()
  if i.strip() == '' or i.strip() =='.' or i.strip() =='...' or i.strip() =='?' or i.strip() =='??' or i.strip() =='love mom':
      continue
  if 'http' in i:
      continue
  else:
      i = i.split('\t')
      if len(i) == 2:
          if i[1] == 'hạnh phúc':
              array_for_label_happy.append(i)
          if i[1] == 'buồn bã':
              array_for_label_sad.append(i)
          if i[1] == 'phẫn nộ':
              array_for_label_fury.append(i)
          if i[1] == 'ngạc nhiên':
              array_for_label_surprise.append(i)
          if i[1] == 'sợ hãi':
              array_for_label_scare.append(i)
          if i[1] == 'thông tin':
              array_for_label_info.append(i)
# new_sen2.extend(array_for_label_happy[0])
new_sen2.extend(array_for_label_happy)
new_sen2.extend(array_for_label_sad)
new_sen2.extend(array_for_label_fury)
new_sen2.extend(array_for_label_surprise)
new_sen2.extend(array_for_label_scare)
new_sen2.extend(array_for_label_info)

In [ ]:
new_sen = combineToList(sentences_1)
# new_sen2 = combineToList(sentences_2)
# new_sen3 = combineToList(sentences_3)

In [ ]:
new_sen.extend(new_sen2)
# print(len(new_sen3))

count_happy = 0
count_sad = 0
count_fury = 0
count_surprise = 0
count_scare = 0
count_info = 0
new_sen_final = []
for i in new_sen:
    if len(i) ==2:
      if i[1] == 'thông tin':
        count_info += 1
      if i[1] == 'hạnh phúc':
          count_happy += 1
      if i[1] == 'phẫn nộ':
          count_fury += 1
      if i[1] == 'ngạc nhiên':
          count_surprise += 1
      if i[1] == 'buồn bã':
          count_sad += 1
      if i[1] == 'sợ hãi':
          count_scare += 1
      new_sen_final.append(i)

# print(len(new_sen_final))
# print('thông tin ' , count_info)
# print('hạnh phúc ' , count_happy)
# print('buồn bã ' , count_sad)
# print('phẫn nộ ' , count_fury)
# print('sợ hãi ' , count_scare)
# print('ngạc nhiên ' , count_surprise)

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/PhoBERT_base_transformers/dict.txt")

2022-08-05 23:41:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [ ]:
def getEndcodeId(sentence):
  ec_sen = bpe.encode(sentence)
  return vocab.encode_line('<s> ' + ec_sen + ' </s>')

In [ ]:
from sklearn.preprocessing import LabelEncoder
train_id, train_text, train_label = [], [], []
test_id, test_text = [], []

for sen in new_sen_final:
  if sen[1] != 'ô' and sen[1] != 'ạ':
    train_label.append(sen[1])
    gettext = chuan_hoa_dau_cau_tieng_viet(sen[0])
    text = rdrsegmenter.tokenize(gettext)
    text = ' '.join([' '.join(x) for x in text])
    train_text.append(text.strip())
    train_id.append(getEndcodeId(text.strip()))


label_encoder = LabelEncoder()
train_label = label_encoder.fit_transform(train_label)

In [ ]:
from sklearn.model_selection import train_test_split
train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_label, test_size=0.2, random_state=42)
print(train_labels)

[2 0 0 ... 1 5 5]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 125

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(

In [ ]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]

    val_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , classification_report
import numpy as np
import time

svm_model = svm.SVC(kernel='linear', probability=True, gamma=0.125) # Linear Kernel

start = time.time()
svm_model.fit(train_masks, train_labels)
end = time.time() - start
y_predict_svm = svm_model.predict(val_masks)
print(classification_report(val_labels,y_predict_svm ,labels=np.unique(y_predict_svm)))

from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000)
start_time = time.time()
LG_model = log_model.fit(train_masks, train_labels)
lr_time = time.time() - start_time
y_predict_lg = LG_model.predict(val_masks)
lr_acc = accuracy_score(val_labels,y_predict_lg)
print(classification_report(val_labels,y_predict_lg ,labels=np.unique(y_predict_lg)))
print(label_encoder.classes_)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       223
           1       0.38      0.87      0.53       585
           3       0.18      0.01      0.01       274
           5       0.32      0.27      0.29       315

   micro avg       0.37      0.43      0.39      1397
   macro avg       0.22      0.29      0.21      1397
weighted avg       0.26      0.43      0.29      1397

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       223
           1       0.38      0.87      0.53       585
           3       0.18      0.01      0.01       274
           5       0.32      0.27      0.29       315

   micro avg       0.37      0.43      0.39      1397
   macro avg       0.22      0.29      0.21      1397
weighted avg       0.26      0.43      0.29      1397

['buồn bã' 'hạnh phúc' 'ngạc nhiên' 'phẫn nộ' 'sợ hãi' 'thông tin']


In [ ]:
sc = svm_model.score(val_masks, val_labels)
print('Train time: ', end)
print('Kết quả train model svm, độ chính xác = ', sc*100, '%')
lg = LG_model.score(val_masks, val_labels)
print('Train time: ', lr_time)
print('Kết quả train model logistic, độ chính xác = ', lg*100, '%')

Train time:  23.633352279663086
Kết quả train model svm, độ chính xác =  36.507936507936506 %
Train time:  5.930971384048462
Kết quả train model logistic, độ chính xác =  36.507936507936506 %


In [ ]:
bert_string = input('Enter sth: ')
MAX_LEN = 125

bert_string = chuan_hoa_dau_cau_tieng_viet(bert_string)
bert_string_arr = []
subwords_string = '<s> ' + bpe.encode(bert_string) + ' </s>'
encoded_sent_string = vocab.encode_line(subwords_string, append_eos=True, add_if_not_exist=False).long().tolist()
bert_string_arr.append(encoded_sent_string)

bert_string_arr = pad_sequences(bert_string_arr, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
bert_inputs_string = torch.tensor(bert_string_arr)
bert_predict_svm = svm_model.predict(bert_inputs_string)
print("SVM tiên đoán: ",label_encoder.inverse_transform(bert_predict_svm)[0])
bert_predict_lg = LG_model.predict(bert_inputs_string)
print("Logistic tiên đoán: ",label_encoder.inverse_transform(bert_predict_lg)[0])

Enter sth: lại 80 triệu à
SVM tiên đoán:  thông tin
Logistic tiên đoán:  thông tin


In [ ]:
# !pip install flask
# !pip install flask_restful
# !pip install flask_cors
# !pip install flask-ngrok

In [ ]:
# from flask import Flask, request, jsonify, json
# from flask_ngrok import run_with_ngrok
# from flask_restful import Resource, Api
# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run

# @app.route("/phobert", methods=['POST'])
# def hello():
#     return "<h1>Hellu</h1>"

# app.run()